# Development of Wastewater Surveillance Data Automation Script 

## 1) Export All data from LIMS DatabasE

In [37]:
#run script that executes export of LIMS data

%run -i "viral_lims_export.py"

####Export all lims data####
df_lims = export_df_from_LIMS()
#df_lims.info()

####Transform lims dataframe#### 
df_lims = rename_lims_columns(df_lims)

####export all dataframes from 4 WW Redcap Projects####
ww_redcap = project_dtype_summary(redcap_api_url, redcap_tokens_prod)

#### Export accepted values in restricted fields from metadata API ####
fields = (
    redcap_metadata_export(redcap_api_url, redcap_tokens_prod["PID171"])
    .pipe(accepted_redcap_fields)
    )

In [ ]:
df_lims.iloc[-15:,:]

# Testing obligate fields import into REDCap

In [94]:
#build a simple dataframe to import obligate value fields only

row = {}
for i,j in fields.items():
    row[i] = next(iter(j))

single_row = pd.DataFrame(row, index = [1])
two_rows = single_row.append(single_row)
two_rows.reset_index(drop = True, inplace=True)
two_rows.index.name = "sample_id"

In [96]:
#Attempt to import two rows
import redcap
project = redcap.Project(redcap_api_url, redcap_tokens_prod["PID171"])

response = project.import_records(two_rows, force_auto_number=False)
response

{'count': 2}

In [126]:
import numpy as np
#setting test data and changing index values
test_01 = df_lims.iloc[-10:].copy()
test_01.index._data = np.array(np.arange(0,10))


In [127]:
#Find which of the obligate value fields are present in the transformed LIMS dataframe
transformed_lims_clms = set(test_01.columns)
obligate_fields = set([i for i in fields.keys()])

obligate_columns = list(obligate_fields & transformed_lims_clms)
test_01 = test_01[obligate_columns].copy()

In [128]:
test_01

,sars_cov2_below_lod,inhibition_adjust,ntc_amplify,concentration_method,quality_flag,extraction_method,sars_cov2_units,inhibition_detect
sample_id,,,,,,,,
0,None,None,No,Skim Milk Flocculation,No,KingFisher,Copies/L,None
1,None,None,No,Skim Milk Flocculation,No,KingFisher,Copies/L,None
2,None,None,No,Skim Milk Flocculation,No,KingFisher,Copies/L,None
3,None,None,No,Skim Milk Flocculation,No,KingFisher,Copies/L,None
4,None,None,No,Skim Milk Flocculation,No,KingFisher,Copies/L,None
5,None,None,No,Skim Milk Flocculation,No,KingFisher,Copies/L,None
6,None,None,No,Skim Milk Flocculation,No,KingFisher,None,None
7,None,None,No,Skim Milk Flocculation,No,KingFisher,None,None
8,None,None,No,Skim Milk Flocculation,No,KingFisher,None,None


###### Devloping function to scan columns for accepted values

In [155]:
# Creating Test datasets
samples_dict = {"column_A": ["A","A", "B", "B", "C","C"],
               "column_B": ["CAT","CAT","Dog","bat","bat","dog"]}

samples_df = pd.DataFrame(samples_dict)


dat_dict = {"column_A": ["A" ,"B"], 
           "column_B": ["CAT", "dog"]}

In [156]:
samples_df

,column_A,column_B
0,A,CAT
1,A,CAT
2,B,Dog
3,B,bat
4,C,bat
5,C,dog


In [157]:
#change values to None if not in list of accepted values, column by column
for i in samples_df.columns:
    
    vals_ok = dat_dict[i] #accepted values for that column
    mask = samples_df[i].isin(vals_ok) #bool check for accepted values
    
    samples_df.loc[~mask, i] = None

In [158]:
samples_df

,column_A,column_B
0,A,CAT
1,A,CAT
2,B,None
3,B,None
4,None,None
5,None,dog


# Developing LIMS Data Validation Function

In [ ]:
lims_clms_validation = {"test_result_date": ["date.today()"],
                        'sample_collect_date':["date.today()"],

                        'sample_collect_time' :["MM:HH"], # must be in #MM:HH format 

                        "pretreatment" :[1,0], # 0 or 1 (yes, no)
                        "sars_cov2_units":[1,2,3,4,5,6], # number 1-6 (drop down categories)
                        "sars_cov2_below_lod":["yes", "no"], # "yes" or "no"
                        "ntc_amplify":["yes", "no"], # "yes" or "no"
                        "inhibition_detect":["yes", "no", "not_tested"], # "yes" or "no" or "not_tested"
                        "inhibition_adjust":["yes", "no"], # "yes" or "no" (only if inhibition_detect = "yes")
                        "concentration_method":[ "mf-mgcl2"], #long list of drop down values
                        "extraction_method":["qiagen-fecal"], #long list of drop down values
                        "quality_flag":["yes", "no"],
                        }

"""
sars_cov2_units  "sars_cov2_units":[1,2,3,4,5,6]
sars_cov2_below_lod "sars_cov2_below_lod":["yes", "no"]
ntc_amplify "ntc_amplify":["yes", "no"]
inhibition_detect "inhibition_detect":["yes", "no", "not_tested"]
inhibition_adjust "inhibition_adjust":["yes", "no"]
concentration_method "concentration_method":[ "mf-mgcl2"]
extraction_method "extraction_method":["qiagen-fecal"]
hum_frac_mic_unit 
other_norm_unit
quality_flag "quality_flag":["yes", "no"]
viral_data_flag

"""

In [ ]:
df_lims["sars_cov2_units"].to_list()

In [ ]:
#shortening df_lims for ease to work with
df_lims_01 = df_lims.reset_index().head().copy()

#enter some test data values
df_lims_01.loc[[0,2],["inhibition_detect"]] = "yes"
df_lims_01.loc[[2],["sars_cov2_units"]] = 4
df_lims_01.loc[[4],["sars_cov2_units"]] = 7


key_01 = "inhibition_detect"
value_01 = lims_clms_validation["inhibition_detect"]

key_01 = "sars_cov2_units"
value_2 = lims_clms_validation["sars_cov2_units"]


a = df_lims_01[key_01].isin(value_01)
b = df_lims_01[key_01].isin(value_01)

In [ ]:
df_lims_01.loc[:,["inhibition_detect", "sars_cov2_units"]]

In [ ]:
lims_clms_validation["sars_cov2_units"]

In [ ]:
df_test_01["test_result_date"] = date.today()
df_test_01['sample_collect_date'] = date.today()
df_test_01['sample_collect_time'] = "10:30"
df_test_01["pretreatment"] = 1 # 0 or 1 (yes, no)
df_test_01["sars_cov2_units"] = 1 # number 1-6 (drop down categories)
df_test_01["sars_cov2_below_lod"] = "yes" # "yes" or "no"
df_test_01["ntc_amplify"] = "yes" # "yes" or "no"
df_test_01["inhibition_detect"] =  "yes" # "yes" or "no" or "not_tested"
df_test_01["inhibition_adjust"] = "yes" # "yes" or "no" (only if inhibition_detect = "yes")
df_test_01["concentration_method"] = "mf-mgcl2" #long list of drop down values
df_test_01["extraction_method"] = "qiagen-fecal" #long list of drop down values
df_test_01["quality_flag"] = "yes" # "yes" or "no"

In [5]:
df = redcap_metadata_export(redcap_api_url, redcap_tokens_prod["PID171"])

In [15]:
df[~df["text_validation_type_or_show_slider_number"].isnull()][["text_validation_type_or_show_slider_number"]]

,text_validation_type_or_show_slider_number
field_name,
label_date,date_mdy
collection_date,date_mdy
sample_collect_date,date_ymd
sample_collect_time,time
ph,number
conductivity,number
collection_water_temp,number
tss,number
cbod,integer


In [8]:
[i for i in df["text_validation_type_or_show_slider_number"]]

[nan,
 nan,
 nan,
 'date_mdy',
 nan,
 nan,
 nan,
 nan,
 'date_mdy',
 nan,
 'date_ymd',
 nan,
 'time',
 nan,
 'number',
 'number',
 'number',
 'number',
 nan,
 nan,
 'integer',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'number',
 'number',
 nan,
 nan,
 'number',
 'date_ymd',
 'number',
 nan,
 nan,
 nan,
 'number',
 'number',
 'number',
 'number',
 nan,
 nan,
 nan,
 nan,
 'number',
 nan,
 nan,
 nan,
 nan,
 'number',
 nan,
 'number',
 nan,
 'number',
 nan,
 nan,
 'number',
 'number',
 nan,
 nan,
 nan,
 nan,
 nan,
 'date_mdy',
 'date_mdy',
 'date_mdy',
 nan,
 nan]

# Apendix

## A-I) Explore datatype stored in LIMS database

pyodbc cursor object allows to interact with database parameters. cursos.columns() returns information about every column in the database table.

In [ ]:
cnxn = pyodbc.connect(credentials) # credentials = 'DSN=LIMS_DATA;UID=xxxxxxx;PWD=xxxxxxx'
cursor = cnxn.cursor()

dtype_list = [(i.column_name, i.type_name) for i in cursor.columns(table="vz_Epi_ELS_SARS-CoV-2 ddPCR")]

dtype_list


RESULT: Two columns have datetime type, remaining columns are varchar type
('TestResultDate', 'datetime')
('SampleCollectDate', 'datetime')

## A-II) Explore converting LIMS dataframe to numeric type - may not be necessary.

In [ ]:
potential_numeric = ["NumNoTargetControl", "SARSCoV2AvgConc"]

In [ ]:
df_lims[potential_numeric] = df_lims[potential_numeric].apply(pd.to_numeric, errors = "coerce")

In [ ]:
df_lims.info()

## B-I) REDCap Manual data export

Exploring manual csv data export - column ID's, Datatypes, Exporting Survey ID and Survey Timestamp

**Conclusion**: 2 additional column are present in when manually exporting csv and keeping survey ID and Survey timestamp selected

In [ ]:
import pandas as pd

#import data
df_PID177_manual = pd.read_csv("./redcap_manual_export/PID177_ww_labs.csv")
df_PID177_manual_noID_noTimeStamp = pd.read_csv("./redcap_manual_export/PID177_ww_labs_minus_SurTimestamp_SurIdentifier.csv")

#make set of column names
columns_PID177_full = set(df_PID177_manual.columns)
columns_PID177_minimal = set(df_PID177_manual_noID_noTimeStamp.columns)

#compare column sets
print("additional columns present: " + str(columns_PID177_full - columns_PID177_minimal))
#print(labs_set_minimal - labs_set_full) #returns empty set 

print("\n")
print(df_PID177_manual[['redcap_survey_identifier', 'a3_ww_lab_set_up_timestamp']])
print("\n")
print(df_PID177_manual.info())

## B-II) REDCap API Data Export

Explore data export via native REDCap API pull

**Conclusion**: API call return data without the additional columns: [redcap_survey_identifier, a3_ww_lab_set_up_timestamp]. These columns can be pulled when exporting data manualy by checking a box.

**Conclusion**: API export columns and manual export columns are identical when survey_identifier and survey_timestamp field remain uncheck during manual export. 

**Conclusion**: During API export, all column fields are objects. Manual export to csv and load to pandas, yields some numeric fields.

**Conclusion**: Datetime format is different between API export, and manual csv export. 

In [ ]:
#export PID177 all data via API
df_PID177_API = redcap_API_export(redcap_api_url, redcap_tokens_prod["PID177"])

API_columns_set = set(df_PID177_API.columns)

#comparing columns of csv manual export with identifier and timestamp fields with standrad API export
print("additional columns present: " + str(columns_PID177_full - API_columns_set))
#print(API_columns_set - columns_PID177_full) #empty set
print()
#are all the columns identical? 
print("Are all the columns identical between standard csv export and API export?")
print(all(df_PID177_manual_noID_noTimeStamp.columns == df_PID177_API.columns))

print()
#converting both manually pulled csv and API data to numberic datatypes (if possible)
#df_PID177_API = df_PID177_API.apply(pd.to_numeric, errors = "ignore")
#df_PID177_manual_noID_noTimeStamp = df_PID177_manual_noID_noTimeStamp.apply(pd.to_numeric, errors = "ignore")

#converting timestamp 
df_PID177_API["ww_lab_setup_date"] = pd.to_datetime(df_PID177_API["ww_lab_setup_date"])
df_PID177_manual_noID_noTimeStamp["ww_lab_setup_date"] = pd.to_datetime(df_PID177_manual_noID_noTimeStamp["ww_lab_setup_date"])
print("after converting all columns to numeric, and 'ww_lab_setup_date' columns to datetime, are the dataframes identical?")

print(df_PID177_manual_noID_noTimeStamp.equals(df_PID177_manual_noID_noTimeStamp))


# B-III) RedCap Pycap Data Export

**Conclusion**: PyCap API calls to export record DO NOT contain record status data

In [ ]:
import redcap
PID177_project = redcap.Project(redcap_api_url, redcap_tokens_prod["PID177"])

In [ ]:
PID177_pycap = PID177_project.export_records(format="df",
                                             event_name = "unique",
                                             #export_checkbox_labels = True,
                                             #export_data_access_groups = True,
                                             #export_survey_fields = True
                                            )



print("Additional columns present: " + str(set(ww_redcap["PID177"].columns) - set(PID177_pycap.columns)))
print(set(PID177_pycap.columns) - set(ww_redcap["PID177"].columns))

# C-I) Exploring LIMS data columns and REDCap data columns

**Conclusion**: Only certain columns from LIMS data map to PID171

**Conclusion**: Created an excel file, to map column name conversion between LIMS and REDCap PID171, created a function to generate dictionary for LIMS data transformation (renaming columns)

In [ ]:
df_WWSamples = ww_redcap["PID171"]
print(df_WWSamples.shape)

In [ ]:
df_lims.dtypes.to_csv("lims_datatypes.csv")
df_WWSamples.dtypes.to_csv("PID171_WW_Samples.csv")

In [ ]:
df_WWSamples.iloc[:,:5]

Function to generate column name transform dictionary. Data lives in xlsx file "LIMS_REDCap_columns_ID's.xlsx"

In [ ]:
df_lims_redcap_column_map = pd.read_excel("LIMS_REDCap_columns_IDs.xlsx", sheet_name = "COMBINED") 

not_null_lims = pd.notna(df_lims_redcap_column_map["LIMS_COLUMNS_NAME"]) #find present LIMS columns that map to REDCap

df_lims_map = df_lims_redcap_column_map[not_null_lims][["PID171_COLUMNS_NAME","LIMS_COLUMNS_NAME"]] #filter for mapped columns

df_lims_map.set_index("LIMS_COLUMNS_NAME", inplace = True)

dict_lims_map = df_lims_map.to_dict()

dict_lims_map = dict_lims_map['PID171_COLUMNS_NAME']

dict_lims_map

# D-I) Demonstration of Upload 1 row of data into REDCap

In [ ]:
from datetime import date

# TEST Import 1 row of data (PID171 format, all blanks)
df_test_01 = ww_redcap["PID171"][df_lims.columns].iloc[0:1].copy()   #take only 1 row of data 


# setting values to df_test
df_test_01.set_index(pd.Series([999999]), inplace = True)  #set index to 999999
df_test_01.loc[999999] = "test"
df_test_01["test_result_date"] = date.today()
df_test_01['sample_collect_date'] = date.today()
df_test_01['sample_collect_time'] = "10:30"
df_test_01["pretreatment"] = 1 # 0 or 1 (yes, no)
df_test_01["sars_cov2_units"] = 1 # number 1-6 (drop down categories)
df_test_01["sars_cov2_below_lod"] = "yes" # "yes" or "no"
df_test_01["ntc_amplify"] = "yes" # "yes" or "no"
df_test_01["inhibition_detect"] =  "yes" # "yes" or "no" or "not_tested"
df_test_01["inhibition_adjust"] = "yes" # "yes" or "no" (only if inhibition_detect = "yes")
df_test_01["concentration_method"] = "mf-mgcl2" #long list of drop down values
df_test_01["extraction_method"] = "qiagen-fecal" #long list of drop down values
df_test_01["quality_flag"] = "yes" # "yes" or "no"



In [ ]:
#Connecting to PID171
import redcap
project = redcap.Project(redcap_api_url, redcap_tokens_prod["PID171"])

response = project.import_records(df_test_01, force_auto_number=False)
response